In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import h5py
drive.mount('/content/drive')
file_path="/content/drive/MyDrive/dataset_ts_light_version.hdf5"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Preprocessing

In [2]:
with h5py.File(file_path, 'r') as f:
  for key in f.keys():
    print(f"{key}: shape={f[key].shape}")

  X_train= f['x_train'][:]
  y_train= f['y_train'][:]

  X_val= f['x_validation'][:]
  y_val= f['y_validation'][:]

  X_test= f['x_test'][:]
  y_test= f['y_test'][:]

x_test: shape=(7766, 48, 48, 3)
x_train: shape=(90601, 48, 48, 3)
x_validation: shape=(31063, 48, 48, 3)
y_test: shape=(7766,)
y_train: shape=(90601,)
y_validation: shape=(31063,)


In [3]:
X = X_train.reshape(X_train.shape[0], -1)
del X_train
means = X.mean(axis=1, keepdims=True)
stds = X.std(axis=1, keepdims=True) + 1e-8
X_standardized = (X-means)/stds
X_standardized[:5]
del X
del means
del stds
# deleting numpy arrays to save RAM

In [4]:
X_standardized.shape

(90601, 6912)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
model_new = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=100)
#10 iter - 0.7841874839041978 accuracy
#20 iter - 0.912052536698429 accuracy
#30 iter - 0.93638938964718 accuracy
#40 iter - 0.9500386299253155 accuracy
#100 iter - 0.9658768993046614 accuracy

In [6]:
model_new.fit(X_standardized, y_train)
# be careful, this reaches 9.5 GB ram used
del X_standardized

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
X = X_test.reshape(X_test.shape[0], -1)
del X_test
means = X.mean(axis=1, keepdims=True)
stds = X.std(axis=1, keepdims=True) + 1e-8
X_test_standardized = (X-means)/stds
X_test_standardized[:5]
del X
del means
del stds
# deleting numpy arrays to save RAM

NameError: name 'X_test' is not defined

In [8]:
y_test_predict = model_new.predict(X_test_standardized)
accuracy = accuracy_score(y_test, y_test_predict)
print(accuracy)

0.9658768993046614


In [9]:
print("Classification Report with Additional Features:")
print(classification_report(y_test, y_test_predict))

Classification Report with Additional Features:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       177
           1       0.98      0.94      0.96       193
           2       0.97      0.93      0.95       195
           3       0.96      0.94      0.95       159
           4       0.97      0.96      0.97       180
           5       0.92      0.91      0.91       170
           6       0.98      0.98      0.98       189
           7       0.96      0.94      0.95       189
           8       0.92      0.95      0.94       213
           9       0.97      0.96      0.97       168
          10       0.96      0.98      0.97       177
          11       0.96      0.93      0.95       184
          12       0.98      0.98      0.98       193
          13       0.99      0.99      0.99       174
          14       0.98      0.98      0.98       185
          15       0.97      0.99      0.98       194
          16       0.98      0.99